In [1]:
# import pandas as pd
from PIL import Image

ImportError: libtiff.so.5: cannot open shared object file: No such file or directory

In [ ]:
fig =  go.Figure()
for file in os.listdir(dir_path):

In [6]:

# file_path = os.path.join(dir_path, file)
df = pd.read_csv('results/level1/beige.csv')
color_scores = df['color_score']
amount_scores = df['amount_score']

# weighted_scores = 
# line_trace = go.Scatter(x=df['iteration'],)

0    0.1
1    0.1
2    0.1
Name: amount_score, dtype: float64

In [13]:
df = pd.read_csv('results/l1-no-refine.csv')

In [22]:
# pd.set_option('display.max_colwidth', -1)
df = pd.read_csv('results/l1-no-refine.csv')
list(df.loc[df['name'] == 'beige.txt', 'llm_response'])[0]

"--- REASONING START ---\nThe target color is a very light shade of orange, which means I need to mix red and green to get orange and then add a significant amount of white to lighten it. Since I don't have white, I can simulate it by using all three primary colors (red, green, blue) in such a way that the mixture is much lighter than the primary colors themselves. The RGB values of the target color are very high and close to each other, with red being the highest, followed by green, and then blue.\n\nTo achieve the target amount of 130ml, I need to plan the proportions of each color carefully. Since the red value is the highest, I will start by adding a large amount of red. The green value is slightly lower, so I will add a little less green than red. The blue value is the lowest, so I will add even less blue.\n\nI will use beaker 3 as my mixing beaker. I will start by adding red and green to get an orange base. Then, I will add a small amount of blue to lighten the color. I will adju

In [2]:
df = pd.DataFrame(columns=['name', 'color_score', 'amount_score', 'llm_response'])

In [6]:
df.loc[len(df)] = ('bruh', 1, 0.9, 'there was once a man')

In [8]:
df.to_csv('results/test', index=False)

In [1]:
# from openai import OpenAI
# client = OpenAI()
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.llms import OpenAI
client = OpenAI()


In [10]:
llm = OpenAI(
    temperature=0.5,
    model_name="gpt-3.5-turbo-instruct"
) 
                             
tools = load_tools(['llm-math'], llm=llm)

agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
    

/home/edwin/miniconda3/envs/nlvr/lib/python3.10/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! system_prompt is not default parameter.
                system_prompt was transferred to model_kwargs.
                Please confirm that system_prompt is what you intended.
  warnings.warn(


In [11]:
print(agent.agent.llm_chain.prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: Calculator\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nALWAYS use the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction:\n```\n$JSON_BLOB\n```\nObservation: the result of the action\n... (this Thought/Action/Observation can repeat N times)\nT

In [15]:
system_prompt="""
You are an agent in a "color-mixing environment". The environment consists 
    of several beakers, each containing a paint of a certain color and amount. 
    Your objective is to mix colors in beakers such that one beaker matches 
    a target beaker as close as possible (in both color and amount).

    At each timestep, you will be given a textual representation of environment
    which tells you the RGB value and amount of paints in each beaker as well
    as the target beaker you are trying to achieve.
    
    Given this description of the envrionment, your task is to generate an 
    action in a specific JSON format. Each action is a JSON object that 
    represents a move in the envivronment. The action should consist of the 
    following elements:

    - "from_beaker": An integer representing the index of the beaker from which the paint is poured.
    - "to_beaker": An integer representing the index of the beaker to which the paint is poured.
    - "transfer_amount": An integer (0 to 100) indicating how much paint (in ml) is transferred from the 'from_beaker' to the 'to_beaker'.
    - "is_done": A boolean value (true or false). Set to true if this is the final action and you want to compare the current state with the target. Set to false otherwise.
    - "compare_beaker": An integer representing the index of the beaker to compare with the target when 'is_done' is true. This field is only relevant if 'is_done' is true.

    Your actions in the environment should work towards producing a beaker
    which is as close as possible to the target beaker (in both color and 
    amount) in as few steps as possible.

    Note that the environment uses a subtractive color mixing model. You should 
    leverage this fact to predict the result of mixing two colors for the 
    purposes of planning. For reference, this is the function used mix colors:

    class SubtractiveModel:
    @staticmethod
    def _rgb_to_cmy(rgb: Tuple[int, int, int]) -> Tuple[int, int, int]:
        return tuple(255 - value for value in rgb)

    @staticmethod
    def _cmy_to_rgb(cmy: Tuple[int, int, int]) -> Tuple[int, int, int]:
        return tuple(255 - value for value in cmy)

    @staticmethod
    def mix_colors(paint1: 'Paint', paint2: 'Paint') -> Tuple[int, int, int]:
        cmy1 = SubtractiveModel._rgb_to_cmy(paint1.color)
        cmy2 = SubtractiveModel._rgb_to_cmy(paint2.color)
        total_amount = paint1.amount + paint2.amount
        if paint1.amount == 0:
            return paint2.color
        if paint2.amount == 0:
            return paint1.color

        mixed_cmy = tuple(int((cmy1[i] * paint1.amount + cmy2[i] * paint2.amount) / total_amount) for i in range(3))
        return SubtractiveModel._cmy_to_rgb(mixed_cmy)
"""

In [22]:
messages = []
messages.append({"role": "system", "content": system_prompt})
obs = """Given the following beakers: 
Beaker 0: RGB(252, 251, 0), 107ml
Beaker 1: RGB(0, 0, 0), 0ml
Beaker 2: RGB(255, 0, 255), 80ml
Beaker 3: RGB(200, 108, 52), 97ml
Beaker 4: RGB(5, 255, 255), 101ml
Target beaker: RGB(44, 220, 226), 63ml
Please output your action given this state."""
messages.append({"role": "user", "content": obs})
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",  
    response_format={"type" : "json_object"},
    messages=messages
)

In [17]:
response.choices[0].message.content

'{\n  "from_beaker": 0,\n  "to_beaker": 1,\n  "transfer_amount": 63,\n  "is_done": true,\n  "compare_beaker": 1\n}'

In [14]:
import json
# Parse the JSON string
action_json = response.choices[0].message.content
action_data = json.loads(action_json)

# Extract action components
from_beaker = action_data.get("from_beaker", 0)
to_beaker = action_data.get("to_beaker", 0)
transfer_amount = action_data.get("transfer_amount", 0)
is_done = action_data.get("is_done", False)
compare_beaker = action_data.get("compare_beaker", 0) if is_done else 0

# Ensure the action components are within the bounds of the action space
# from_beaker = min(max(from_beaker, 0), action_space.nvec[0] - 1)
# to_beaker = min(max(to_beaker, 0), action_space.nvec[1] - 1)
# transfer_amount = min(max(transfer_amount, 0), action_space.nvec[2] - 1)
# is_done = 1 if is_done else 0
# compare_beaker = min(max(compare_beaker, 0), action_space.nvec[4] - 1)

# Construct the action tuple
action = (from_beaker, to_beaker, transfer_amount, is_done, compare_beaker)
print(action)

(0, 1, 63, True, 1)
